# ---- Targeted Adversarial Attacks

In [1]:
#----Tensorflow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
#----Other packages
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

In [2]:
im_path = '.../pig.jpg'
image = cv2.imread(im_path)

### Define a function that gets the class label and returns the integer class label index for that label
(from the "imagenet_class_index.json file)

In [3]:
#get json file
def get_class_idx(label):
    labelPath = '...imagenet_class_index.json'
    with open(labelPath) as f:
        #Loop through all the labels and indexes and put in dictionary
        imageNetClasses = {labels[1]:int(idx)for (idx, labels) in json.load(f).items()}
    # return the value(idx) that we are looking for with the label
    return imageNetClasses.get(label, None)

### Define the preprocess funcion for the image so is properly feed to Resnet50 required dimensions

In [4]:
def preprocess_image(images):
    image = cv2.cvtColor(images,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))
    image = np.expand_dims(image, axis=0)
    return image

### Define some variables for the RESNET50 model and initialize it

In [5]:
#Epsilon: vaue used to clipping tensonrs when constructing an adversarial images
#It adds the noise in a way that the image doesn't change to the human eye
EPS = 2/255.0 #standard values used in research
LR = 5e-2

In [6]:
#Initialize the model
model = ResNet50(weights='imagenet')
optimizer = Adam(learning_rate=LR)
sccLoss = SparseCategoricalCrossentropy()

2022-09-20 16:45:19.479112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:45:19.487130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:45:19.487346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:45:19.487938: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Prepare the image for the adversarial process and the noise vector (delta)

In [7]:
image = preprocess_image(image)
baseImage = tf.constant(image, dtype= tf.float32)
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

### Implementing the adversarial images attacks

In [8]:
#clip the values of the tensor to a given range
def clip_eps(tensor, eps):
    return tf.clip_by_value(tensor, clip_value_min = -eps, clip_value_max=eps)

In [9]:
def generate_targeted_adversaries(model, baseImage, delta, classIdx, target, steps=500):
    for step in range(0, steps):
        with tf.GradientTape() as tape:
            #track the gradient updates from the perturbation vector
            tape.watch(delta)
            #Add the perturbation vector to the base Image
            adversary = preprocess_input(baseImage + delta)

            #Use this new image and pass it through the model to calculate the 2 losses
            #The first (negative) to tell the model to incorrectly classify the origianl class
            #The second to reinforce the wrong label as the correct class label
            predictions = model(adversary, training=False)
            originalLoss = -sccLoss(tf.convert_to_tensor([classIdx]), predictions)
            targetLoss = sccLoss(tf.convert_to_tensor([target]), predictions)
            #obtain the sum between the losses so themodel doens't deviate
            totalLoss = originalLoss + targetLoss

            #Check the logs of the losses we are obtaining
            if step%20==0:
                print("step: {}, loss: {}".format(step, totalLoss.numpy()))

        #calculate the gradients repex to the perturbaation loss
        gradients = tape.gradient(totalLoss, delta)

        #update weights, clip perturbation  and update values
        optimizer.apply_gradients([(gradients, delta)])
        delta.assign_add(clip_eps(delta, eps=EPS))
    #return perturbation vector
    return delta

In [10]:
#run it. We are targetting for the "Lakeland_terrier" label
deltaUpdated = generate_targeted_adversaries(model, baseImage, delta, 341, 189)

2022-09-20 16:45:21.678352: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-09-20 16:45:21.923700: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-20 16:45:21.924493: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-20 16:45:21.924525: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-09-20 16:45:21.925402: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-20 16:45:21.925529: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


step: 0, loss: 27.60276985168457
step: 20, loss: 5.422225475311279
step: 40, loss: -1.5594053268432617
step: 60, loss: -6.942905902862549
step: 80, loss: -13.461017608642578
step: 100, loss: -25.787336349487305
step: 120, loss: -53.72135543823242
step: 140, loss: -86.631103515625
step: 160, loss: -119.08494567871094
step: 180, loss: -147.3900909423828
step: 200, loss: -172.62832641601562
step: 220, loss: -192.90736389160156
step: 240, loss: -209.41485595703125
step: 260, loss: -224.27487182617188
step: 280, loss: -237.79188537597656
step: 300, loss: -248.9915008544922
step: 320, loss: -259.1815185546875
step: 340, loss: -267.68475341796875
step: 360, loss: -275.1045837402344
step: 380, loss: -282.2637023925781
step: 400, loss: -288.83575439453125
step: 420, loss: -295.16943359375
step: 440, loss: -301.21160888671875
step: 460, loss: -306.998291015625
step: 480, loss: -312.92999267578125


### Create the adversarial image, return chnnaels to BGR and save image

In [11]:
adverImage = (baseImage + deltaUpdated).numpy().squeeze()
adverImage = np.clip(adverImage, 0, 255).astype("uint8")
adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
cv2.imwrite('.../pig_targeted.jpg', adverImage)

True

### Get the results of the predictions

In [12]:
preprocessedImage = preprocess_input(baseImage + deltaUpdated)
predictions = model.predict(preprocessedImage)
predictions = decode_predictions(predictions, top=3)[0]
label = predictions[0][1]
confidence = predictions[0][2] * 100
print("[INFO] label: {} confidence: {:.2f}%".format(label,
	confidence))

1/1 [==============================] - 1s 939ms/step
[INFO] label: Lakeland_terrier confidence: 100.00%


In [13]:
text = "{}: {:.2f}%".format(label, confidence)
cv2.putText(adverImage, text, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
	(0, 255, 0), 2)

# show the output image
cv2.imshow("Output", adverImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to target thread (0x558eb8620e00)

QObject::moveToThread: Current thread (0x558eb8620e00) is not the object's thread (0x558ed7983730).
Cannot move to tar